## Omega and Xi

To implement Graph SLAM, a matrix and a vector (omega and xi, respectively) are introduced. The matrix is square and labelled with all the robot poses (xi) and all the landmarks (Li). Every time you make an observation, for example, as you move between two poses by some distance `dx` and can relate those two positions, you can represent this as a numerical relationship in these matrices.

Below you can see a matrix representation of omega and a vector representation of xi.

<img src='P3_Implement_SLAM-master/images/omega_xi.png' width=20% height=20% />


### Solving for x, L

To "solve" for all these poses and landmark positions, we can use linear algebra; all the positional values are in the vector `mu` which can be calculated as a product of the inverse of omega times xi.

---

## 오메가와 Xi

그래프 SLAM을 구현하기 위해 행렬과 벡터(각각 오메가와 xi)를 도입합니다. 이 행렬은 정사각 행렬이며 모든 로봇 포즈(xi)와 모든 랜드마크(Li)로 표시됩니다. 예를 들어, 두 포즈 사이를 일정 거리 `dx`만큼 이동하면서 관찰을 하고 두 위치를 연관시킬 때마다, 이를 이러한 행렬의 수치적 관계로 표현할 수 있습니다.

아래는 오메가를 행렬로 표현한 것과 xi를 벡터로 표현한 것입니다.

<img src='P3_Implement_SLAM-master/images/omega_xi.png' width=20% height=20% />

### x, L에 대한 풀이

이러한 모든 포즈와 랜드마크 위치를 "풀기" 위해 선형 대수를 사용할 수 있습니다. 모든 위치 값은 벡터 `mu`에 있으며, 이는 오메가에 xi를 곱한 역수의 곱으로 계산할 수 있습니다.

---


### Quiz: Construct constraints for 3 motions and return `mu`

In the following example, you will complete the function call `mu_from_positions(-3, 5, 3)`, which takes in 3 robot poses/moves:
* initial pose: -3
* moves by 5
* moves by 3

In this function, you should construct the constraint matrices `omega` and `xi` and calculate `mu`. The final call should result in a `mu` of:
```
[[-3.0],
 [2.0],
 [5.0]]
 ```

 ### 퀴즈: 3가지 동작에 대한 제약 조건을 생성하고 `mu`를 반환합니다.

다음 예제에서는 3가지 로봇 포즈/동작을 받는 `mu_from_positions(-3, 5, 3)` 함수 호출을 완료합니다.
* 초기 포즈: -3
* 5만큼 이동합니다.
* 3만큼 이동합니다.

이 함수에서 제약 조건 행렬 `omega`와 `xi`를 생성하고 `mu`를 계산해야 합니다. 최종 호출 결과는 다음과 같은 `mu`입니다.
```
[[-3.0],
[2.0],
[5.0]]
```

## Constraint Updates (X)

We will not consider landmark sensor measurements in this example, only robot poses.

#### Motion
When your robot moves by some amount `dx` update the constraint matrices as follows:
* Add `[[1, -1], [-1, 1]]` to omega at the indices for the intersection of `xt` and `xt+1`
* Add `-dx` and `dx` to xi at the rows for `xt` and `xt+1`

## 제약 조건 업데이트 (X)

이 예제에서는 랜드마크 센서 측정값을 고려하지 않고 로봇 자세만 고려합니다.

#### 동작
로봇이 일정량 `dx`만큼 이동하면 제약 조건 행렬을 다음과 같이 업데이트합니다.
* `xt`와 `xt+1`의 교점에 대한 인덱스에서 오메가에 `[[1, -1], [-1, 1]]`을 더합니다.
* `xt`와 `xt+1`의 행에서 xi에 `-dx`와 `dx`를 더합니다.

In [1]:
import numpy as np

def mu_from_positions(initial_pos, move1, move2):

    ## TODO: construct constraint matrices
    ## and add each position/motion constraint to them
    omega = np.zeros((3, 3))
    xi = np.zeros((3, 1))

    # Your code here
    # Initial state - first position
    omega[0, 0] = 1
    xi[0] = initial_pos

    # Move 1
    omega += [
        [1, -1, 0],
        [-1, 1, 0],
        [0, 0, 0]
    ]
    xi +=[
        [-move1],
        [move1],
        [0]
    ]

    # Move 2
    omega += [
        [0, 0, 0],
        [0, 1, -1],
        [0, -1, 1]
    ]
    xi +=[
        [0],
        [-move2],
        [move2]
    ]


    # display final omega and xi
    print('Omega: \n', omega)
    print('\n')
    print('Xi: \n', xi)
    print('\n')

    ## TODO: calculate mu as the inverse of omega * xi
    ## recommended that you use: np.linalg.inv(np.matrix(omega)) to calculate the inverse
    mu = np.linalg.inv(np.matrix(omega))*xi
    return mu


In [2]:
# call function and print out `mu`
mu = mu_from_positions(-3, 5, 3)
print('Mu: \n', mu)

Omega: 
 [[ 2. -1.  0.]
 [-1.  2. -1.]
 [ 0. -1.  1.]]


Xi: 
 [[-8.]
 [ 2.]
 [ 3.]]


Mu: 
 [[-3.]
 [ 2.]
 [ 5.]]


## Constraint Updates (2)

In the below code, we construct `omega` and `xi` constraint matrices, and update these according to landmark sensor measurements and motion.

#### Sensor Measurements

When you sense a distance, `dl`, between a pose and a landmark, l, update the constraint matrices as follows:
* Add `[[1, -1], [-1, 1]]` to omega at the indices for the intersection of `xt` and `l`
* Add `-dl` and `dl` to xi at the rows for `xt` and `l`

The values 2 instead of 1 indicate the "strength" of the measurement.

You'll see three new `dl`'s as new inputs to our function `Z0, Z1, Z2`, below.

#### Motion
When your robot moves by some amount `dx` update the constraint matrices as follows:
* Add `[[1, -1], [-1, 1]]` to omega at the indices for the intersection of `xt` and `xt+1`
* Add `-dx` and `dx` to xi at the rows for `xt` and `xt+1`

## QUIZ: Include three new sensor measurements for a single landmark, L.

## 제약 조건 업데이트 (2)

아래 코드에서는 `omega`와 `xi` 제약 조건 행렬을 구성하고 랜드마크 센서 측정값과 움직임에 따라 이를 업데이트합니다.

#### 센서 측정값

포즈와 랜드마크 l 사이의 거리 `dl`을 감지하면 다음과 같이 제약 조건 행렬을 업데이트합니다.
* `xt`와 `l`의 교차점 인덱스에서 omega에 `[[1, -1], [-1, 1]]`을 더합니다.
* `xt`와 `l`의 행에서 xi에 `-dl`과 `dl`을 더합니다.

1 대신 2인 값은 측정의 "강도"를 나타냅니다.

아래 함수 `Z0, Z1, Z2`에 대한 새로운 입력으로 세 개의 새로운 `dl`이 표시됩니다.

#### 동작
로봇이 일정량 `dx`만큼 움직일 때 제약 조건 행렬을 다음과 같이 업데이트합니다.
* `xt`와 `xt+1`의 교점에 대한 인덱스에서 오메가에 `[[1, -1], [-1, 1]]`을 더합니다.
* `xt`와 `xt+1`의 행에서 xi에 `-dx`와 `dx`를 더합니다.

## 퀴즈: 단일 랜드마크 L에 대한 세 가지 새로운 센서 측정값을 포함합니다.


In [27]:
import numpy as np


def mu_from_positions(initial_pos, move1, move2, Z0, Z1, Z2):

    ## TODO: construct constraint matrices
    ## and add each position/motion constraint to them

    # initialize constraint matrices with 0's
    # Now these are 4x4 because of 3 poses and a landmark
    omega = np.zeros((4, 4))
    xi = np.zeros((4, 1))

    # Your code here
    # account for the first motion, dx = move1
    omega[0, 0] = 1
    xi[0] = initial_pos

    # Move 1
     # Move 1
    omega += [
        [1, -1, 0,0],
        [-1, 1, 0,0],
        [0, 0, 0,0],
        [0, 0, 0,0]
    ]
    xi +=[
        [-move1],
        [move1],
        [0],
        [0]
    ]

      # Move 2
    omega += [
        [0, 0, 0,0],
        [0, 1, -1,0],
        [0, -1, 1,0],
        [0, 0, 0,0]
    ]
    xi +=[
        [0],
        [-move2],
        [move2],
        [0],
        
    ]

    # SENSE
    omega += [
        [1, 0, 0, -1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [-1, 0, 0, 1]
    ]
    xi +=[
        [-Z0],
        [0],
        [0],
        [Z0]
    ]

    omega += [
        [0, 0, 0, 0],
        [0, 1, 0, -1],
        [0, 0, 0, 0],
        [0, -1, 0, 1]
    ]
    xi +=[
        [0],
        [-Z1],
        [0],
        [Z1]
    ]

    omega += [
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 1, -1],
        [0, 0, -1, 1]
    ]
    xi +=[
        [0],
        [0],
        [-Z2],
        [Z2]
    ]



    ## TODO: Include three new sensor measurements for the landmark, L
    # Your code here
    # Landmark measurement 1


    # display final omega and xi
    print('Omega: \n', omega)
    print('\n')
    print('Xi: \n', xi)
    print('\n')

    ## TODO: calculate mu as the inverse of omega * xi
    ## recommended that you use: np.linalg.inv(np.matrix(omega)) to calculate the inverse
    mu = np.linalg.inv(np.matrix(omega))*xi
    return mu

In [28]:
# call function and print out `mu`
mu = mu_from_positions(-3, 5, 3, 10, 5, 2)
print('Mu: \n', mu)

Omega: 
 [[ 3. -1.  0. -1.]
 [-1.  3. -1. -1.]
 [ 0. -1.  2. -1.]
 [-1. -1. -1.  3.]]


Xi: 
 [[-18.]
 [ -3.]
 [  1.]
 [ 17.]]


Mu: 
 [[-3.]
 [ 2.]
 [ 5.]
 [ 7.]]


## Constraint Updates (3)

In the below code, we construct `omega` and `xi` constraint matrices, and update these according to landmark sensor measurements and motion.

#### Sensor Measurements

When you sense a distance, `dl`, between a pose and a landmark, l, update the constraint matrices as follows:
* Add `[[1, -1], [-1, 1]]` to omega at the indices for the intersection of `xt` and `l`
* Add `-dl` and `dl` to xi at the rows for `xt` and `l`

The values 2 instead of 1 indicate the "strength" of the measurement.

You'll see three new `dl`'s as new inputs to our function `Z0, Z1, Z2`, below.

#### Motion
When your robot moves by some amount `dx` update the constraint matrices as follows:
* Add `[[1, -1], [-1, 1]]` to omega at the indices for the intersection of `xt` and `xt+1`
* Add `-dx` and `dx` to xi at the rows for `xt` and `xt+1`

## Change the code so that the last sensor measurement (Z2) is extremely confident.

Multiple the "strength" of that measurement by a factor of 5.

## 제약 조건 업데이트 (3)

아래 코드에서는 `omega`와 `xi` 제약 조건 행렬을 구성하고 랜드마크 센서 측정값과 움직임에 따라 이를 업데이트합니다.

#### 센서 측정값

포즈와 랜드마크 l 사이의 거리 `dl`을 감지하면 다음과 같이 제약 조건 행렬을 업데이트합니다.
* `xt`와 `l`의 교차점 인덱스에서 omega에 `[[1, -1], [-1, 1]]`을 더합니다.
* `xt`와 `l`의 행에서 xi에 `-dl`과 `dl`을 더합니다.

1 대신 2인 값은 측정의 "강도"를 나타냅니다.

아래 함수 `Z0, Z1, Z2`에 대한 새로운 입력으로 세 개의 새로운 `dl`이 표시됩니다.

#### 동작
로봇이 일정량 `dx`만큼 움직일 때 제약 조건 행렬을 다음과 같이 업데이트합니다.
* `xt`와 `xt+1`의 교점에 있는 인덱스에서 omega에 `[[1, -1], [-1, 1]]`을 더합니다.
* `xt`와 `xt+1`의 행에서 xi에 `-dx`와 `dx`를 더합니다.

## 마지막 센서 측정값(Z2)이 매우 신뢰할 수 있도록 코드를 변경합니다.

해당 측정값의 "강도"를 5배로 곱합니다.

In [36]:
import numpy as np


def mu_from_positions(initial_pos, move1, move2, Z0, Z1, Z2):

    ## TODO: construct constraint matrices
    ## and add each position/motion constraint to them

    # initialize constraint matrices with 0's
    # Now these are 4x4 because of 3 poses and a landmark
    omega = np.zeros((4,4))
    xi = np.zeros((4,1))

    # add initial pose constraint
    omega[0][0] = 1
    xi[0] = initial_pos

    # account for the first motion, dx = move1
    omega += [[1., -1., 0., 0.],
              [-1., 1., 0., 0.],
              [0., 0., 0., 0.],
              [0., 0., 0., 0.]]
    xi += [[-move1],
           [move1],
           [0.],
           [0.]]

    # account for the second motion
    omega += [[0., 0., 0., 0.],
              [0., 1., -1., 0.],
              [0., -1., 1., 0.],
              [0., 0., 0., 0.]]
    xi += [[0.],
           [-move2],
           [move2],
           [0.]
           
           ]

    ## TODO: Include three new sensor measurements for the landmark, L
    # incorporate first sense
    omega += [
        [1, 0, 0, -1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [-1, 0, 0, 1]
    ]
    xi +=[
        [-Z0],
        [0],
        [0],
        [Z0]
    ]

    omega += [
        [0, 0, 0, 0],
        [0, 1, 0, -1],
        [0, 0, 0, 0],
        [0, -1, 0, 1]
    ]
    xi +=[
        [0],
        [-Z1],
        [0],
        [Z1]
    ]

    omega += [
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 5, -5],
        [0, 0, -5, 5]
    ]
    xi +=[
        [0],
        [0],
        [-Z2],
        [Z2]
    ]


    # incorporate second sense


    ## This third sense is now *very confident* and
    ## we multiply everything by a strength factor of 5 instead of 1
    # incorporate third sense


    # display final omega and xi
    print('Omega: \n', omega)
    print('\n')
    print('Xi: \n', xi)
    print('\n')

    ## TODO: calculate mu as the inverse of omega * xi
    ## recommended that you use: np.linalg.inv(np.matrix(omega)) to calculate the inverse
    mu = np.linalg.inv(np.matrix(omega))*xi
    return mu


In [37]:
# call function and print out `mu`
mu = mu_from_positions(-3, 5, 3, 10, 5, 1)
print('Mu: \n', mu)

Omega: 
 [[ 3. -1.  0. -1.]
 [-1.  3. -1. -1.]
 [ 0. -1.  6. -5.]
 [-1. -1. -5.  7.]]


Xi: 
 [[-18.]
 [ -3.]
 [  2.]
 [ 16.]]


Mu: 
 [[-3.        ]
 [ 2.32142857]
 [ 6.28571429]
 [ 6.67857143]]
